# Klassifikation von Pistazien


## Installations skript

In [16]:
""" Downloaden des Pistazien Datensatzes """


C:\Users\benni\.cache\kagglehub\datasets\muratkokludataset\pistachio-dataset\versions\1


## Vorbereitung des Datensatzes
Um mit dem Datensatz besser arbeiten zu können, werden folgende Schritte durchlaufen:
- Es wird sich zuerst angeschaut wie der Datensatz aufgebaut ist. → [Explorative Datenanalyse](#Explorative-Datenanalyse)
- Anschließend muss der Datensatz ggf. bereinigt/aufbereitet werden. → [Aufbereitung der Daten](#Aufbereitung-der-Daten)
- Zuletzt wird der Datensatz noch in Trainings- und Testdaten aufgeteilt. → [Aufteilung der Daten in Trainings- und Testdaten](#Aufteilung-der-Daten-in-Trainings--und-Testdaten)

Danach kann es mit der Klassifikation losgehen. → [Klassifikation](#Klassifikation)

### Explorative Datenanalyse

### Aufbereitung der Daten

### Aufteilung der Daten in Trainings- und Testdaten


## Klassifikation